# MAST 查询

**用例：** 能够对NIRSpec数据执行MAST查询。<br>

**数据：** 无

**工具：** astroquery。<br>

**跨仪器：** 所有仪器。<br>

**文档：** 此笔记本是STScI更大[后处理数据分析工具生态系统](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis)的一部分。<br>

在本教程结束时，您将能够：

1) 执行针对NIRSpec数据的MAST查询

2) 理解NIRSpec MOS、IFU和固定光缝数据的MAST搜索选项

3) 按各种参数过滤MAST查询

4) 对您的目标进行潜在重复性检查

## 引言

MAST（巴巴拉·A·米库尔斯基太空望远镜档案）提供了多种方法来检索数据。[MAST门户](https://outerspace.stsci.edu/display/MASTDOCS/Portal+Guide)允许用户以交互方式搜索特定观测并按关键词过滤。为帮助用户以这种方式访问MAST，提供了详细的[教程](https://outerspace.stsci.edu/display/MASTDOCS/Portal+Guide)。在某些情况下，特别是对于大型查询，使用[Astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html)可能更为高效。文档中描述了一些广泛的功能，包括位置查询、元数据查询和目录查询。关于[MAST API](https://mast.stsci.edu/api/v0/)的一般文档也可用。

用户可能希望搜索档案的一个原因是寻找其提案的潜在重复观测。如[JWST重复观测政策](https://jwst-docs.stsci.edu/jwst-opportunities-and-policies/jwst-general-science-policies/jwst-duplicate-observations-policy)中所述，如果观测使用相同的仪器观察相同的天文源或领域，并且具有相似的灵敏度和光谱范围，则该观测可能是另一观测的重复。有关更具体的仪器细节，请参阅文档。

本笔记本涵盖以下主题：

1) 如何使用Python提交NIRSpec MAST查询

    - 如何对所有NIRSpec数据进行广泛搜索

    - 如何使用一个查询过滤参数进行搜索

    - 如何使用多个查询过滤参数进行搜索

    - 可用搜索参数的信息

2) 如何通过与现有MAST数据进行比较，检查任何给定目标的潜在重复问题（注意：这可能不包括已安排的观测，结果应在APT中确认）

    - 定义描述潜在观测的输入目录

    - 在搜索半径内查询MAST，以获取具有重叠波长和相似曝光时间的观测

    - 检查潜在的重复观测

尽管本笔记本是为NIRSpec用户设计的，但其他JWST仪器的MAST查询结构类似。有关其他仪器的更多相关信息，请参阅[Astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html)网页。

## 主要内容

In [ ]:
# 导入MAST库
from astroquery.mast import Mast  # 从astroquery库中导入Mast模块，用于访问MAST数据

### 执行所有 NIRSpec 数据的搜索

这个第一个示例演示了如何使用 [Mast.service_request_async()](https://astroquery.readthedocs.io/en/latest/api/astroquery.mast.MastClass.html#astroquery.mast.MastClass.service_request_async)，该方法基于服务和参数构建并执行一个 [Mashup](https://mast.stsci.edu/api/v0/class_mashup_1_1_mashup_request.html) 查询。有关更多信息和选项描述，请参见 [文档](https://mast.stsci.edu/api/v0/class_mashup_1_1_mashup_request.html)。这种格式的响应提供了一种可以列出文件名的格式。

In [ ]:
# 设置要查询的仪器服务，格式为首字母大写

service = "Mast.Jwst.Filtered.Nirspec"

# 输入查询参数，或者像这里一样留空以检索该仪器的所有数据。

# 将 'columns' 留为空星号将包括所有数据列。将 'columns' 替换为 COUNT_BIG(*) 将 

#     返回搜索中的文件数量。

parameters = {"columns": "*",  # 查询所有列

               "filters": []}  # 过滤器为空，表示不使用任何过滤条件

# 检索 MAST 响应（此步骤可能需要几秒钟）

response = Mast.service_request_async(service, parameters)  # 异步请求服务并获取响应

In [ ]:
# 收集结果

results = response[0].json()['data']  # 从响应中提取数据部分

In [ ]:
# 从每个结果中提取文件名，如果你想要一个文件名列表

filenames = []  # 初始化一个空列表，用于存储文件名

for result in results:  # 遍历结果列表中的每个结果

    filename = result['filename'].split('.')[0]  # 获取文件名并去掉扩展名

    filenames.append(filename)  # 将处理后的文件名添加到列表中

In [ ]:
# 打印前十个文件名
print(filenames[:10])  # 输出filenames列表中的前十个元素

### 使用一个查询参数：

In [ ]:
# 输入一个查询参数：

service = "Mast.Jwst.Filtered.Nirspec"  # 设置服务名称为JWST NIRSpec过滤服务

one_parameter = {"columns": "*",  # 查询所有列

               "filters": [{"paramName": "exp_type",  # 设置过滤条件，参数名称为exp_type

                            "values": ['NRS_MSASPEC']  # 过滤值为NRS_MSASPEC

                            }]

             }

response_one_parameter = Mast.service_request_async(service, one_parameter)  # 异步请求服务

results_one_parameter = response_one_parameter[0].json()['data']  # 获取返回结果中的数据部分

# 打印前十个文件名

for result_one_parameter in results_one_parameter[:10]:  # 遍历前十个结果

    print(result_one_parameter['filename'].split('.')[0])  # 打印文件名（去掉扩展名）

### 使用多个查询参数：

In [ ]:
# 使用多个查询参数

service = "Mast.Jwst.Filtered.Nirspec"  # 设置服务名称

multiple_parameters = {
    "columns": "*",  # 查询所有列

    "filters": [
        {"paramName": "apername",  # 过滤器参数：光圈名称
         "values": ['NRS1_FULL', 'NRS_FULL_MSA']  # 允许的光圈名称值
        },
        {"paramName": "detector",  # 过滤器参数：探测器
         "values": ['NRS1']  # 允许的探测器值
        },
        {"paramName": "filter",  # 过滤器参数：滤光片
         "values": ['F290LP', 'F170LP']  # 允许的滤光片值
        },
        {"paramName": "exp_type",  # 过滤器参数：曝光类型
         "values": ['NRS_IFU', 'NRS_MSASPEC', 'NRS_BRIGHTOBJ', 'NRS_WATA']  # 允许的曝光类型值
        },
        {"paramName": "readpatt",  # 过滤器参数：读取模式
         "values": ['NRS', 'NRSRAPID']  # 允许的读取模式值
        }
    ]
}

response_multiple_parameters = Mast.service_request_async(service, multiple_parameters)  # 异步请求服务

results_multiple_parameters = response_multiple_parameters[0].json()['data']  # 获取返回的JSON数据

# 打印前十个文件名
for result_multiple_parameters in results_multiple_parameters[:10]:
    # 除了文件名，还可以打印其他参数。下面是可用选项的列表。
    print(result_multiple_parameters['filename'].split('.')[0] + '     ' + result_multiple_parameters['visitype'])  # 打印文件名和可视类型

### 可用的过滤参数列表在档案中：

以下参数可用于MAST查询服务的过滤器。要使用这些过滤器，请将过滤器作为'paramName'添加，并将您希望过滤的各种选项作为'values'。

除了用作过滤器之外，如果您希望返回除文件名以外的内容，任何这些键也可以用来访问或返回结果的任何组成部分。例如，您可以通过将"result['filename']"替换为"result['pi_name']"来打印PI名称，而不是打印文件名。

In [ ]:
# 所有NIRSpec数据的字典键（不带任何滤镜）：

all_keys = ""  # 初始化一个空字符串，用于存储所有键

for result in results:  # 遍历结果列表中的每个结果

    for key in result.keys():  # 遍历当前结果的所有键

        if key not in all_keys:  # 如果当前键不在all_keys中

            all_keys = all_keys + key + ", "  # 将键添加到all_keys中，并加上逗号和空格

print(all_keys[:-2])  # 打印所有键，去掉最后的逗号和空格

注意，上述某些参数是特定于某种观测模式的。例如，'msaconid'、'msametfl'、'msametid' 和 'msastate' 对应于多目标光谱（MOS）观测。

In [ ]:
# 存储给定参数（例如，exp_type）的所有选项
exp_type_list = []

# 遍历结果集
for x in results:
    # 如果当前的exp_type不在exp_type_list中，则添加它
    exp_type_list.append(x['exp_type']) if x['exp_type'] not in exp_type_list else exp_type_list

# 遍历所有唯一的exp_type并打印
for exp_type in exp_type_list:
    print(exp_type)

## 潜在重复检查

NIRSpec的重复观测可能发生在重复搜索半径内存在一个目标，该目标不仅使用相同的光栅（grating）和色散器（disperser），而且其曝光时间（exposure time）与之前观测的曝光时间相差不超过四倍。在宽场成像中，视场重叠（field overlap）必须大于50%才能被视为重复观测。有关[识别潜在重复观测](https://jwst-docs.stsci.edu/jwst-duplication-checking/identifying-potential-duplicate-observations)的更多信息可能对观测者有所帮助。

在此演示中，使用输入目录（input catalog）搜索一个半径等于光圈最长边的区域，查询那些符合波长（wavelength）和曝光时间标准的观测，并在档案中找到可能是目录中任何目标的重复观测的源。

虽然上面提到的[Mast.service_request_async()](https://astroquery.readthedocs.io/en/latest/api/astroquery.mast.MastClass.html#astroquery.mast.MastClass.service_request_async)方法对于使用元数据参数过滤数据非常有用，但在这种情况下使用[Observations.query_criteria()](https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries)来搜索特定的天空区域，并附加其他参数。

重复搜索半径依赖于光圈大小，这些光圈在[NIRSpec固定光缝JDox页面](https://jwst-docs.stsci.edu/near-infrared-spectrograph/nirspec-instrumentation/nirspec-fixed-slits#:~:text=NIRSpec%20has%20five%20fixed%20slits,series%20(BOTS)%20observing%20mode)中进行了描述。光缝的最长边用于固定光缝的重复搜索半径。以下单元格包含每个NIRSpec光圈的重复搜索半径（以度为单位）。

In [ ]:
# 默认重复搜索半径

mos_radius = 180/360  # 将角秒转换为度，180角秒/360
mos_slit_radius = 0.46/360 # 将角秒转换为度，0.46角秒/360
ifu_radius = 3/360  # 将角秒转换为度，3角秒/360
fs_S1600A1_radius = 1.6/360  # 将角秒转换为度，1.6角秒/360
fs_S200A1_radius = 3.3/360  # 将角秒转换为度，3.3角秒/360
fs_S200A2_radius = 3.3/360  # 将角秒转换为度，3.3角秒/360
fs_S400A1_radius = 3.8/360  # 将角秒转换为度，3.8角秒/360
fs_S200B1_radius = 3.3/360  # 将角秒转换为度，3.3角秒/360

在潜在重复观测搜索中使用的搜索区域由上述定义的孔径的最长边决定。

In [ ]:
# 定义一个字典，包含不同观测区域的半径
search_area = {
    "MSA": mos_radius,          # MSA区域的半径
    "IFU": ifu_radius,          # IFU区域的半径
    "S1600A1": fs_S1600A1_radius,  # S1600A1区域的半径
    "S200A1": fs_S200A1_radius,    # S200A1区域的半径
    "S200A2": fs_S200A2_radius,    # S200A2区域的半径
    "S400A1": fs_S400A1_radius,    # S400A1区域的半径
    "S200B1": fs_S200B1_radius      # S200B1区域的半径
}

在这种情况下，标记为 'instrument_name' 的值包括仪器和模式。此字典用于确定在查询中使用的适当名称。

In [ ]:
# 定义一个字典，包含不同仪器名称及其对应的描述
instrument_name = {
    "MSA": "NIRSPEC/MSA",  # 多目标光谱仪
    "IFU": "NIRSPEC/IFU",  # 介质光纤光谱仪
    "S1600A1": "NIRSPEC/SLIT",  # 光纤光谱仪 S1600A1
    "S200A1": "NIRSPEC/SLIT",  # 光纤光谱仪 S200A1
    "S200A2": "NIRSPEC/SLIT",  # 光纤光谱仪 S200A2
    "S400A1": "NIRSPEC/SLIT",  # 光纤光谱仪 S400A1
    "S200B1": "NIRSPEC/SLIT"   # 光纤光谱仪 S200B1
}

设置输入目录，包括目标目录号、光圈（aperture）、赤经（RA）、赤纬（DEC）、光栅（grating）、滤光片（filter）、曝光时间（exposure time）、搜索区域（search area）以及每个目标的仪器/模式名称（instrument/mode name）。选择这种格式是为了方便将额外的目标轻松添加到输入目录列表中。尽管这些是进行重复检查的主要考虑因素，但也可以使用这种字典格式添加其他标准。使用Astroquery检查哪些[观测场](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html)可用于过滤NIRSpec数据可能会很有用。

请注意，NIRSpec FS或IFU光谱观测可以被视为与相同天文目标的MOS观测在相似波长范围和分辨率下的重复。因此，在搜索FS或IFU观测潜在重复时，查询MOS观测也可能会有所帮助。

In [ ]:
# 输入用于查询的目标目录

target1_number = "1"  # 目标1的编号
target1_aperture = "MSA"  # 目标1的光圈类型
target1_RA = 53.13  # 目标1的右升角
target1_DEC = -27.8  # 目标1的 declination（赤纬）
target1_grating = "G395H"  # 目标1的光栅类型
target1_filters = "F290LP"  # 目标1的滤光片
target1_exposure_time = 950  # 目标1的曝光时间
target1_search_area = search_area[target1_aperture]  # 目标1的搜索区域
target1_instrument_name = instrument_name[target1_aperture]  # 目标1的仪器名称
target1 = {'number': target1_number, 'RA': target1_RA, 'DEC': target1_DEC, 'grating': target1_grating, 
           'filters': target1_filters, 'exposure_time': target1_exposure_time, 'search_area': target1_search_area,
           'instrument_name': target1_instrument_name}  # 目标1的字典信息

target2_number = "2"  # 目标2的编号
target2_aperture = "IFU"  # 目标2的光圈类型
target2_RA = 68.73091  # 目标2的右升角
target2_DEC = 24.48140  # 目标2的 declination（赤纬）
target2_grating = "G395H"  # 目标2的光栅类型
target2_filters = "F290LP"  # 目标2的滤光片
target2_exposure_time = 22.749  # 目标2的曝光时间
target2_search_area = search_area[target2_aperture]  # 目标2的搜索区域
target2_instrument_name = instrument_name[target2_aperture]  # 目标2的仪器名称
target2 = {'number': target2_number, 'RA': target2_RA, 'DEC': target2_DEC, 'grating': target2_grating,
           'filters': target2_filters, 'exposure_time': target2_exposure_time, 'search_area': target2_search_area,
           'instrument_name': target2_instrument_name}  # 目标2的字典信息

target3_number = "3"  # 目标3的编号
target3_aperture = "S200B1"  # 目标3的光圈类型
target3_RA = 5.130  # 目标3的右升角
target3_DEC = -36.895  # 目标3的 declination（赤纬）
target3_grating = "G140H"  # 目标3的光栅类型
target3_filters = "F070LP"  # 目标3的滤光片
target3_exposure_time = 1880.22  # 目标3的曝光时间
target3_search_area = search_area[target3_aperture]  # 目标3的搜索区域
target3_instrument_name = instrument_name[target3_aperture]  # 目标3的仪器名称
target3 = {'number': target3_number, 'RA': target3_RA, 'DEC': target3_DEC, 'grating': target3_grating,
           'filters': target3_filters, 'exposure_time': target3_exposure_time, 'search_area': target3_search_area,
           'instrument_name': target3_instrument_name}  # 目标3的字典信息

input_catalog = [target1, target2, target3]  # 输入目录包含目标1、目标2和目标3

导入 [MAST 的 Observations 包](https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries)，以便根据特定条件进行查询，而不仅仅是基于位置或目标名称。该包还允许您通过 'proposal_pi' 或其他 [观测字段](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html) 进行搜索。在重复检查的情况下，这种查询格式特别有用，因为它允许用户在最小和最大 RA（赤经）和 Dec（赤纬）值范围内进行搜索，此外还可以使用过滤器和光栅等其他参数。

In [ ]:
from astroquery.mast import Observations  # 从astroquery库导入Observations模块，用于查询MAST数据
import numpy as np  # 导入numpy库，用于数值计算

接下来，使用目标的目录编号作为键生成一个字典，该字典包含输入目录中每个目标的观测表。

In [ ]:
# 生成观测表的字典

table = {}  # 初始化一个空字典，用于存储观测结果

for target in input_catalog:  # 遍历输入目录中的所有目标

    key = target['number']  # 获取目标的编号作为字典的键

    # 使用输入目录中的信息执行查询

    table[key] = Observations.query_criteria(instrument_name=target['instrument_name'],  # 查询仪器名称
                     s_ra=[(target['RA'] - target['search_area']), (target['RA'] + target['search_area'])],  # 查询右升角范围
                     s_dec=[target['DEC'] - target['search_area'], target['DEC'] + target['search_area']],  # 查询天赤纬范围
                     grating=target['grating'],  # 查询光栅
                     filters=target['filters'],  # 查询滤光片
                     t_exptime=[(target['exposure_time']) / 4, (target['exposure_time']) * 4])  # 查询曝光时间范围

    # 对于MOS数据执行查询，因为FS和IFU观测可能是MOS观测的重复

    if target['instrument_name'] != "NIRSPEC/MSA":  # 如果仪器不是NIRSPEC/MSA

        table[key + "MOS"] = Observations.query_criteria(instrument_name="NIRSPEC/MSA",  # 查询NIRSPEC/MSA仪器
                         s_ra=[target['RA'] - target['search_area'], target['RA'] + target['search_area']],  # 查询右升角范围
                         s_dec=[target['DEC'] - target['search_area'], target['DEC'] + target['search_area']],  # 查询天赤纬范围
                         grating=target['grating'],  # 查询光栅
                         filters=target['filters'],  # 查询滤光片
                         t_exptime=[(target['exposure_time']) / 4, (target['exposure_time']) * 4]  # 查询曝光时间范围
                         )

    print("""The observation table below contains potential duplicate observations for 

          target {} in the input catalog.""".format(key))  # 打印目标编号的提示信息

    print(table[key][:5])  # 打印观测表的前五条结果（可以去掉[:5]以显示所有结果）

    print()  # 打印空行以分隔输出

因为在APT中检查这些观测是很重要的，以更可靠地确定这些潜在的重复观测是否确实被视为重复，因此可以使用此工具来检索提案ID，然后在APT中进行搜索。

In [ ]:
# 打印完整表格中生成的提案ID列表

proposal_ids=[]  # 初始化一个空列表以存储提案ID

for target in input_catalog:  # 遍历输入目录中的每个目标

    for proposal_id in table[target['number']]['proposal_id']:  # 遍历每个观测的提案ID

        if int(proposal_id) not in proposal_ids:  # 检查提案ID是否已存在以去除重复

            proposal_ids.append(int(proposal_id))  # 将唯一的提案ID添加到列表中

print("It may be helpful to check these proposal IDs in APT to compare proposal details:", proposal_ids)  # 打印提案ID列表以供参考

In [ ]:
# 获取产品组ID列表（使用obsid而不是观察ID / obs_id）

obs_ids=[]  # 初始化一个空列表以存储唯一的obsid

for target in input_catalog:  # 遍历输入目录中的每个目标

    for obs_id in table[target['number']]['obsid']:  # 遍历每个观察的提案ID

        if obs_id not in obs_ids:  # 检查obs_id是否已存在于列表中以去除重复项

            obs_ids.append(obs_id)  # 如果不存在，则将其添加到列表中

print('Product group IDs:', obs_ids[:10])  # 打印前10个产品组ID

In [ ]:
# 根据产品组ID（例如 '2003839997'）下载数据产品

# 数据将下载到当前工作目录下名为 "mastDownload" 的文件夹，除非作为参数提供了下载目录。

# 警告：此单元格可能需要几分钟才能运行。

product_group_ID = '2003839997'  # 定义产品组ID

manifest = Observations.download_products(product_group_ID, download_dir=None)  # 下载产品并返回清单

请注意，如果您想访问专有数据，您可能需要先登录。更多信息可以在文档中找到，链接为 [访问专有数据](https://astroquery.readthedocs.io/en/v0.3.7/mast/mast.html#accessing-proprietary-data)。

### 额外资源

有一个[笔记本](https://github.com/spacetelescope/JWST_Planned_Observations/blob/master/Querying_JWST_in_astroquery.ipynb)可用，它通过查询位置展示了目标区域。另一个笔记本提供了关于[Mashup](https://github.com/spacetelescope/JWST_Planned_Observations/blob/master/Querying_JWST_in_mashup.ipynb)的详细信息。这些笔记本可能对更一般的JWST重复检查有用，因为它们提供了上传目标文件列表或在[Aladin](https://aladin.u-strasbg.fr/)中可视化结果等信息。

正如在[识别潜在重复观测](https://jwst-docs.stsci.edu/jwst-duplication-checking/identifying-potential-duplicate-observations#IdentifyingPotentialDuplicateObservations-Portaltargetsearch)中更详细地指出的，检查这些目标在[APT](https://www.stsci.edu/scientific-community/software/astronomers-proposal-tool-apt)中是至关重要的。

如果您使用astroquery，请引用论文[Ginsburg, Sipőcz, Brasseur等 2019](https://ui.adsabs.harvard.edu/abs/2019AJ....157...98G/abstract)。

#### 关于这个笔记本：

***作者：*** Teagan King，科学支持分析师

***更新时间：*** 2021年3月12日

看起来您发送的内容为空。如果您有特定的Markdown内容需要翻译，请提供文本，我将很高兴为您翻译并保持格式。

[页面顶部](#top)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="太空望远镜标志" width="200px"/>